### Library import and code configuration

In [75]:
import numpy as np
import pandas as pd
import geocoder

In [2]:
pd.options.display.float_format = '{:.2f}'.format

Geocoder

In [39]:
def geoc(loc):
    g = geocoder.google(loc)
    g = g.json
    print (loc)
    if len(g) != 2 or g is None:
        g = geocoder.arcgis(loc)
        g = g.json

        if len(g) != 2 or g is None:
            g = geocoder.geocodefarm(loc)
            g = g.json

            if len(g) != 2 or g is None:
                g = geocoder.osm(loc)
                g = g.json

                if len(g) != 2 or g is None:
                    g = geocoder.yandex(loc)
                    g = g.json
    else:
        g = None
    
    print(g)
    return g

In [37]:
def getAirlines(txt):
    airlines = ['aena','aeromar','aeromexico','aireuropa','airmadrid','airnostrum','americanairlines',
                       'avianca','blueair','britishairways','easyjet','emirates','eurowings','flybe','iberia','klm',
                       'lufthansa','niki','norwegian','qatar','ryanair','spanair','spiritairlines','tame','vasp',
                       'volaris','vueling','westjet','wizzair']
    
    arl = ''
    
    for airline in airlines:
        if airline.lower() in txt.lower():
            arl = airline
        
    return arl

### Dataframe

In [96]:
fileIn = '/media/dedsresende/BackUpDeds/Github_UB/CapstoneProject/Feelings/kagle_capstone_project/presentacion/data/tweets_public_decode.csv'
fileOut = '/media/dedsresende/BackUpDeds/Github_UB/CapstoneProject/Feelings/kagle_capstone_project/presentacion/data/tweets_public_clean.csv'

In [6]:
df = pd.read_csv(fileIn, encoding = 'utf-16')
df.head()

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,positive,False,0,0,"""Los pilotos de Ryanair desconvocan la huelga ...",NaN,Mon Dec 18 13:07:04 +0000 2017,942743012337122944.00,NaN,Dublin
1,positive,True,0,0,"@Iberia @lavecinarubia Si ,por favor las decla...",NaN,Sat Nov 04 17:05:11 +0000 2017,926857871916184064.00,NaN,NaN
2,neutral,True,0,0,@Iberia Me dirías por favor que costo tiene?,NaN,Sat Dec 02 15:24:09 +0000 2017,936979305720091008.00,NaN,NaN
3,negative,True,0,0,"@SupermanlopezN @Iberia @giroditalia Champion,...",NaN,Thu Dec 21 23:17:43 +0000 2017,943983853802328064.00,NaN,Central Time (US & Canada)
4,negative,True,0,0,@SrtaFarrellDM @KLM @Iberia Eso de avianca es ...,NaN,Wed Dec 06 00:44:25 +0000 2017,938207464457212032.00,NaN,Eastern Time (US & Canada)


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7867 entries, 0 to 7866
Data columns (total 10 columns):
airline_sentiment    7867 non-null object
is_reply             7867 non-null bool
reply_count          7867 non-null int64
retweet_count        7867 non-null int64
text                 7867 non-null object
tweet_coord          22 non-null object
tweet_created        7867 non-null object
tweet_id             7867 non-null float64
tweet_location       439 non-null object
user_timezone        5119 non-null object
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 560.9+ KB


In [8]:
df['tweet_id'] = df['tweet_id'].astype('category')
df['airline_sentiment'] = df['airline_sentiment'].astype('category')
df['retweet_count'] = df['retweet_count'].astype('int64')
df['text'] = df['text'].astype('str')
df['tweet_coord'] = df['tweet_coord'].astype('str')
df['tweet_created'] = pd.to_datetime(df['tweet_created'])
df['tweet_location'] = df['tweet_location'].astype('category')
df['user_timezone'] = df['user_timezone'].astype('category')

In [9]:
df['lat'] = df['tweet_coord'].apply(lambda x: np.nan if x == 'nan' else float((x.replace('[',' ').replace(']',' ').replace(' ','').split(','))[0]))
df['lon'] = df['tweet_coord'].apply(lambda x: np.nan if x == 'nan' else float((x.replace('[',' ').replace(']',' ').replace(' ','').split(','))[1]))

In [10]:
df['airline'] = df['text'].apply(lambda x: getAirlines(x))

In [50]:
ttloc = {}

for loc in df[df['tweet_location'].notnull()]['tweet_location'].unique():
    g = geocoder.google(loc.split(',')[0])
    latlng = g.latlng
    
    if latlng is None:
        g = geocoder.arcgis(loc.split(',')[0])
        latlng = g.latlng

        if latlng is None:
            g = geocoder.geocodefarm(loc.split(',')[0])
            latlng = g.latlng

            if latlng is None:
                g = geocoder.osm(loc.split(',')[0])
                latlng = g.latlng

                if latlng is None:
                    g = geocoder.yandex(loc.split(',')[0])
                    latlng = g.latlng
    
    try:
        ttloc[loc] = {'lat': latlng[0], 'lng': latlng[1]}
        print(ttloc[loc])
        
    except Exception as e:
        print(e)

{'lat': 40.42028000000005, 'lng': -3.7057699999999727}
{'lat': -34.459329999999966, 'lng': -58.91528999999997}
{'lat': 1.209510000000023, 'lng': -77.27664999999996}
{'lat': 41.651830000000075, 'lng': -0.8811399999999594}
{'lat': 40.6400629, 'lng': 22.9444191}
{'lat': 40.42028000000005, 'lng': -3.7057699999999727}
{'lat': 43.529742, 'lng': 5.447426999999999}
{'lat': 19.4306069, 'lng': -99.0949899}
{'lat': 39.571480000000065, 'lng': 2.646940000000029}
{'lat': -33.41656999999998, 'lng': -70.59552999999994}
{'lat': 40.42028000000005, 'lng': -3.7057699999999727}
{'lat': 31.0811365, 'lng': -97.36748399999999}
{'lat': 43.3069814, 'lng': -2.9126022}
{'lat': 10.56737000000004, 'lng': -84.00272999999999}
{'lat': -34.6036844, 'lng': -58.3815591}
{'lat': 41.3120158, 'lng': 2.0959429}
{'lat': -1.831239, 'lng': -78.18340599999999}
{'lat': 50.1109221, 'lng': 8.6821267}
{'lat': 41.38561000000004, 'lng': 2.1687200000000644}
{'lat': 10.3910485, 'lng': -75.4794257}
{'lat': 41.10934890000001, 'lng': 1.140

{'lat': 41.38561000000004, 'lng': 2.1687200000000644}
{'lat': 34.0522342, 'lng': -118.2436849}
{'lat': 5.534420000000068, 'lng': -73.36149999999998}
{'lat': 51.5465063, 'lng': -0.1058058}
{'lat': 48.754020000000025, 'lng': 2.215370000000064}
{'lat': 36.84191000000004, 'lng': -2.463619999999935}
{'lat': 41.14946000000003, 'lng': -8.61030999999997}
{'lat': 19.3794059, 'lng': -99.15914459999999}
{'lat': 36.71333000000004, 'lng': 3.212500000000034}
{'lat': -34.52114999999998, 'lng': -58.47307999999998}
{'lat': 43.55899000000005, 'lng': -5.921479999999974}
{'lat': 40.052490000000034, 'lng': 0.05927000000002636}
{'lat': 51.1078852, 'lng': 17.0385376}
{'lat': -33.023909999999944, 'lng': -71.55216999999993}
{'lat': 48.71299000000005, 'lng': 2.3657100000000355}
{'lat': 38.72639000000004, 'lng': -9.149479999999926}
{'lat': 52.51604000000003, 'lng': 13.376910000000066}
{'lat': 13.3387613, 'lng': -81.3729372}
{'lat': 41.7735409, 'lng': 12.2397117}
{'lat': 40.426210000000026, 'lng': -3.563989999999

In [51]:
usrloc = {}

for loc in df[df['user_timezone'].notnull()]['user_timezone'].unique():
    g = geocoder.google(loc.split(',')[0])
    latlng = g.latlng
    
    if latlng is None:
        g = geocoder.arcgis(loc.split(',')[0])
        latlng = g.latlng

        if latlng is None:
            g = geocoder.geocodefarm(loc.split(',')[0])
            latlng = g.latlng

            if latlng is None:
                g = geocoder.osm(loc.split(',')[0])
                latlng = g.latlng

                if latlng is None:
                    g = geocoder.yandex(loc.split(',')[0])
                    latlng = g.latlng
    
    try:
        usrloc[loc] = {'lat': latlng[0], 'lng': latlng[1]}
        print(usrloc[loc])
        
    except Exception as e:
        print(e)

{'lat': 53.3498053, 'lng': -6.2603097}
{'lat': 37.09024, 'lng': -95.712891}
{'lat': 37.51647579999999, 'lng': -82.8067126}
{'lat': 24.8607343, 'lng': 67.0011364}
{'lat': 37.09024, 'lng': -95.712891}
{'lat': 40.4167754, 'lng': -3.7037902}
{'lat': 19.4326077, 'lng': -99.133208}
{'lat': 10.4805937, 'lng': -66.90360629999999}
{'lat': 37.9838096, 'lng': 23.7275388}
{'lat': -34.6036844, 'lng': -58.3815591}
{'lat': -33.43721999999997, 'lng': -70.65001999999998}
{'lat': 71.706936, 'lng': -42.604303}
{'lat': 52.3702157, 'lng': 4.895167900000001}
{'lat': 52.51604000000003, 'lng': 13.376910000000066}
{'lat': 45.8150108, 'lng': 15.9819189}
{'lat': 4.710988599999999, 'lng': -74.072092}
{'lat': 48.856614, 'lng': 2.3522219}
{'lat': 51.5073509, 'lng': -0.1277583}
{'lat': 60.16985569999999, 'lng': 24.9383791}
{'lat': 28.2072168, 'lng': -177.3734926}
{'lat': 46.0569465, 'lng': 14.5057515}
{'lat': -34.6036844, 'lng': -58.3815591}
{'lat': 37.09024, 'lng': -95.712891}
{'lat': 43.6383714, 'lng': -79.421133}

In [57]:
df['tweet_location_coord'] = df['tweet_location'].map(ttloc)
df['user_timezone_coord'] = df['user_timezone'].map(usrloc)

In [71]:
df['tweet_location_coord'] = df['tweet_location_coord'].astype('object')
df['user_timezone_coord'] = df['user_timezone_coord'].astype('object')

In [83]:
df['tt_lat'] = df['tweet_location_coord'].apply(lambda x: None if type(x) is float else x['lat'])
df['tt_lng'] = df['tweet_location_coord'].apply(lambda x: None if type(x) is float else x['lng'])

In [84]:
df['usr_lat'] = df['user_timezone_coord'].apply(lambda x: None if type(x) is float else x['lat'])
df['usr_lng'] = df['user_timezone_coord'].apply(lambda x: None if type(x) is float else x['lng'])

In [90]:
df['hour'] = df['tweet_created'].dt.hour
df['date'] = df['tweet_created'].dt.date
df['dow'] = df['tweet_created'].dt.dayofweek

In [92]:
df['airline'] = df['text'].apply(lambda x: getAirlines(x))

In [93]:
df.head()

,airline_sentiment,is_reply,reply_count,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone,...,airline,tweet_location_coord,user_timezone_coord,tt_lat,tt_lng,usr_lat,usr_lng,hour,date,dow
0,positive,False,0,0,"""Los pilotos de Ryanair desconvocan la huelga ...",nan,2017-12-18 13:07:04,942743012337122944.00,NaN,Dublin,...,ryanair,NaN,"{'lat': 53.3498053, 'lng': -6.2603097}",nan,nan,53.35,-6.26,13,2017-12-18,0
1,positive,True,0,0,"@Iberia @lavecinarubia Si ,por favor las decla...",nan,2017-11-04 17:05:11,926857871916184064.00,NaN,NaN,...,iberia,NaN,NaN,nan,nan,nan,nan,17,2017-11-04,5
2,neutral,True,0,0,@Iberia Me dirías por favor que costo tiene?,nan,2017-12-02 15:24:09,936979305720091008.00,NaN,NaN,...,iberia,NaN,NaN,nan,nan,nan,nan,15,2017-12-02,5
3,negative,True,0,0,"@SupermanlopezN @Iberia @giroditalia Champion,...",nan,2017-12-21 23:17:43,943983853802328064.00,NaN,Central Time (US & Canada),...,iberia,NaN,"{'lat': 37.09024, 'lng': -95.712891}",nan,nan,37.09,-95.71,23,2017-12-21,3
4,negative,True,0,0,@SrtaFarrellDM @KLM @Iberia Eso de avianca es ...,nan,2017-12-06 00:44:25,938207464457212032.00,NaN,Eastern Time (US & Canada),...,klm,NaN,"{'lat': 37.51647579999999, 'lng': -82.8067126}",nan,nan,37.52,-82.81,0,2017-12-06,2


In [94]:
dfExport = df[['airline_sentiment', 'airline', 'lat', 'lon', 'tweet_location', 'tt_lat', 'tt_lng','user_timezone', 'usr_lat', 'usr_lng','hour','date','dow']]

In [101]:
dfExport.to_csv(path_or_buf=fileOut, sep=',', na_rep='', header=True, index=False, encoding='utf-8', mode='w', quotechar='"', line_terminator='\n', decimal='.')

### Dashboard

Colors

In [9]:
rd = '#E4523B'
dkbl = '#0A454D'
ltbl = '#3DB296'
ltyl = '#ECC417'
dkyl = '#E8931E'
cz = '#969696'

In [10]:
colors = {'negative':rd, 'positive':ltbl, 'neutral':cz}
df['sent_colors'] = df['airline_sentiment'].map(colors)

Control configs

In [11]:
TOOLS = "tap,box_select,lasso_select,pan,wheel_zoom,box_zoom,reset"

Labels

In [12]:
tltPlot = figure(x_range=(1, 9), y_range=(20, 100), plot_width=1200, plot_height=100)
tltPlot.xgrid.visible = False
tltPlot.ygrid.visible = False
tltPlot.axis.visible = False
tltPlot.outline_line_color = None

label = Label(x=1, y=25, text='Capstone Project: Feelings', text_font_size='36pt', text_color='black')


tltPlot.add_layout(label)

In [13]:
lblPlot1 = figure(x_range=(1, 9), y_range=(20, 100), plot_width=400, plot_height=100, title = 'Negative tweets')
lblPlot1.xgrid.visible = False
lblPlot1.ygrid.visible = False
lblPlot1.axis.visible = False
lblPlot1.title.text_color = "black"
lblPlot1.outline_line_color = None

label1 = Label(x=1, y=18, text=str(df[df['airline_sentiment']=='negative']['airline_sentiment'].size), text_font_size='36pt', text_color=rd)


lblPlot1.add_layout(label1)

In [14]:
lblPlot2 = figure(x_range=(1, 9), y_range=(20, 100), plot_width=400, plot_height=100, title = 'Positive tweets')
lblPlot2.xgrid.visible = False
lblPlot2.ygrid.visible = False
lblPlot2.axis.visible = False
lblPlot2.title.text_color = "black"
lblPlot2.outline_line_color = None

label2 = Label(x=1, y=18, text=str(df[df['airline_sentiment']=='positive']['airline_sentiment'].size), text_font_size='36pt', text_color=ltbl)

lblPlot2.add_layout(label2)

In [15]:
lblPlot3 = figure(x_range=(1, 9), y_range=(20, 100), plot_width=400, plot_height=100, title = 'Neutral tweets')
lblPlot3.xgrid.visible = False
lblPlot3.ygrid.visible = False
lblPlot3.axis.visible = False
lblPlot3.title.text_color = "black"
lblPlot3.outline_line_color = None

label3 = Label(x=1, y=18, text=str(df[df['airline_sentiment']=='neutral']['airline_sentiment'].size), text_font_size='36pt', text_color=cz)

lblPlot3.add_layout(label3)

Map configs

In [16]:
plotMapBase = figure(
    x_range=(df['lon3857'].min()-2500000, df['lon3857'].max()+2500000),
    y_range=(df['lat3857'].min()-2500000, df['lat3857'].max()+2500000),
    plot_width=1200,
    plot_height=600,
    tools = TOOLS,
    title = 'Where are the tweets comming from'
)

plotMapBase.title.text_color = "black"
plotMapBase.add_tile(STAMEN_TONER)
plotMapBase.xgrid.visible = False
plotMapBase.ygrid.visible = False
plotMapBase.axis.visible = False

Data sources

In [17]:
source = ColumnDataSource(
    data = dict(
        lat = df['lat3857'],
        lng = df['lon3857'],
        sent = df['airline_sentiment'],
        cl = df['sent_colors']
    )
)

Map markers

In [18]:
mapMarker = Circle(x='lng', y='lat', size=5, fill_color='cl', fill_alpha=0.5, line_color='cl')
plotMapBase.add_glyph(source, mapMarker)

GlyphRenderer(id='f651d428-8e2c-4ba2-b453-1bb2c0ee271f', ...)

Bar chart airlines sentiment

In [19]:
dfAirlines = df.groupby(['airline', 'airline_sentiment'])['airline', 'airline_sentiment'].size().reset_index()

dfAirlines.columns = ['airline','airline_sentiment','number']

dfAirlines['airline'] = dfAirlines['airline'].astype('str')
dfAirlines['airline_sentiment'] = dfAirlines['airline_sentiment'].astype('str')

dfAirlines['x'] = dfAirlines['airline']+' '+dfAirlines['airline_sentiment']

barSource = ColumnDataSource(
    data = dict(
        x = dfAirlines['x'],
        arln = dfAirlines['airline'],
        snt = dfAirlines['airline_sentiment'],
        n = dfAirlines['number'],
        col = dfAirlines['airline_sentiment'].map(colors)
    )
)

In [20]:
barSentiment = figure(x_range=FactorRange(*dfAirlines['x']), plot_height=200, plot_width = 1200, title="Sentiment by Airlines")

barSentiment.vbar(x='x', top='n', width=0.9, source=barSource, line_color="white",fill_color='col')

barSentiment.y_range.start = 0
barSentiment.x_range.range_padding = 0.1
barSentiment.xaxis.major_label_orientation = pi/4
barSentiment.xgrid.visible = False
barSentiment.ygrid.visible = False
barSentiment.outline_line_color = None

Time line

In [21]:
dfDate = df.groupby([df['tweet_created'].dt.date, df['airline_sentiment']]).size().reset_index()
dfDate.columns = ['date','airline_sentiment','number']

dateSource = ColumnDataSource(
    data = dict(
        x = dfDate[dfDate['airline_sentiment']=='negative']['date'],
        yNeg = dfDate[dfDate['airline_sentiment']=='negative']['number'],
        yPos = dfDate[dfDate['airline_sentiment']=='positive']['number'],
        yNtr = dfDate[dfDate['airline_sentiment']=='neutral']['number']
    )
)

In [22]:
dateSentiment = figure(x_axis_type="datetime", plot_height=200, plot_width = 600, title="Sentiment by time")

dateSentiment.circle('x', 'yNeg', source=dateSource, color=rd)
dateSentiment.line('x', 'yNeg', source=dateSource, color=rd)

dateSentiment.circle('x', 'yPos', source=dateSource, color=dkbl)
dateSentiment.line('x', 'yPos', source=dateSource, color=dkbl)

dateSentiment.circle('x', 'yNtr', source=dateSource, color=cz)
dateSentiment.line('x', 'yNtr', source=dateSource, color=cz)

dateSentiment.xgrid.visible = False
dateSentiment.ygrid.visible = False
dateSentiment.outline_line_color = None

In [23]:
dfHour = df.groupby([df['tweet_created'].dt.hour, df['airline_sentiment']]).size().reset_index()
dfHour.columns = ['hour','airline_sentiment','number']

hourSource = ColumnDataSource(
    data = dict(
        x = dfHour[dfHour['airline_sentiment']=='negative']['hour'],
        yNeg = dfHour[dfHour['airline_sentiment']=='negative']['number'],
        yPos = dfHour[dfHour['airline_sentiment']=='positive']['number'],
        yNtr = dfHour[dfHour['airline_sentiment']=='neutral']['number']
    )
)

In [24]:
hourSentiment = figure(x_axis_type="datetime", plot_height=200, plot_width = 600, title="Sentiments during the day")

hourSentiment.circle('x', 'yNeg', source=hourSource, color=rd)
hourSentiment.line('x', 'yNeg', source=hourSource, color=rd)

hourSentiment.circle('x', 'yPos', source=hourSource, color=dkbl)
hourSentiment.line('x', 'yPos', source=hourSource, color=dkbl)

hourSentiment.circle('x', 'yNtr', source=hourSource, color=cz)
hourSentiment.line('x', 'yNtr', source=hourSource, color=cz)

hourSentiment.xgrid.visible = False
hourSentiment.ygrid.visible = False
hourSentiment.outline_line_color = None

Plot

In [26]:
p = gridplot([[tltPlot],[lblPlot1, lblPlot2, lblPlot3],[plotMapBase],[barSentiment],[dateSentiment,hourSentiment]])
show(p)
# output_file("tourist_dashboard.html")
# save(p)

W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='3855b863-5ce6-423c-8669-4d742bdacd5e', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='90bb4725-0ba3-40dd-9814-24da56dacc3f', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='9f30295a-6cee-4528-b281-46f440e8c6b0', ...)
W-1001 (NO_DATA_RENDERERS): Plot has no data renderers: Figure(id='ba23609b-f653-47ce-b068-9d69e24ae79f', ...)
